In [3]:
!pip install konlpy
!pip install nltk

불용어 직접 추가

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def load_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []

    i = 0
    while i < len(lines):
        speaker_line = lines[i].strip()
        content_line = lines[i+1].strip() if i+1 < len(lines) else ""

        # 발화자 정보가 'Speaker'로 시작하는지 확인
        if speaker_line.startswith('Speaker'):
            speaker_info = speaker_line.split()
            if len(speaker_info) >= 3:
                speaker = speaker_info[1]
                time = speaker_info[2]

                data.append([speaker, time, content_line])

        i += 2

    df = pd.DataFrame(data, columns=['Speaker', 'Time', 'Content'])
    return df

# 사용자 정의 한국어 불용어 목록
korean_stop_words = [
    '그', '그래서', '그러나', '그리고', '이', '저', '것', '등', '더', '수', '등등', '하지만', '그리고', '다시', '때문에', '이제', '저희', '그런', '저희가', '제가', '근데', '너무', '이런', '이렇게', '저는', '저도', '좋아요', '하고', '하는', '혹시'
]

# 데이터 로드 (원하는 파일 경로로 수정)
file_path = 'transcriptions/project4/transcription_project4_09.txt'
df = load_and_preprocess(file_path)

# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer(max_features=10, stop_words=korean_stop_words)  # 최대 10개의 키워드를 추출

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

# 키워드 확인
keywords = tfidf_vectorizer.get_feature_names_out()
print("키워드:", keywords)

# TF-IDF 행렬을 DataFrame으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=keywords)

# 원본 데이터와 결합
df = pd.concat([df, tfidf_df], axis=1)

# 각 참가자의 키워드 빈도 합계 계산
keyword_freq_by_speaker = df.groupby('Speaker')[keywords].sum()

# 중요 발언자 선정
important_speakers = keyword_freq_by_speaker.sum(axis=1).sort_values(ascending=False).index.tolist()

print("중요 발언자 순위:")
for speaker in important_speakers:
    print(f"{speaker}: {keyword_freq_by_speaker.loc[speaker].sum()}")

print("\n키워드 빈도 데이터프레임:")
print(keyword_freq_by_speaker)

키워드: ['그게' '그냥' '그러니까' '그럼' '아니' '우리가' '이거' '이게' '있는' '지금']
중요 발언자 순위:
SPEAKER_03: 137.62333108296826
SPEAKER_00: 62.55116139915662
SPEAKER_01: 45.53221583225188
SPEAKER_04: 35.57577190386088
SPEAKER_02: 2.0

키워드 빈도 데이터프레임:
                  그게         그냥       그러니까        그럼         아니       우리가  \
Speaker                                                                     
SPEAKER_00  2.476825   8.573756   6.876987  6.831557   6.760044  5.918792   
SPEAKER_01  4.773924   7.694423  11.672553  4.516351   1.000000  1.745748   
SPEAKER_02  0.000000   1.000000   0.000000  0.000000   0.000000  0.000000   
SPEAKER_03  8.478022  13.947390  15.806804  3.808463  11.432373  9.688905   
SPEAKER_04  3.000000   3.897552   1.000000  7.712852   3.000000  3.751569   

                   이거         이게        있는         지금  
Speaker                                                
SPEAKER_00   7.255762   6.094259  6.126800   5.636379  
SPEAKER_01   3.548846   1.698652  3.260756   5.620962  
SPEAKER_02  

불용어 파일 사용

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

# 한국어 불용어 목록
stopwords_file = 'data/stopwords-ko.txt'

with open(stopwords_file, 'r', encoding='utf-8') as f:
    stopwords_list = [word.strip() for word in f.readlines()]

def load_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []

    i = 0
    while i < len(lines):
        speaker_line = lines[i].strip()
        content_line = lines[i+1].strip() if i+1 < len(lines) else ""

        # 발화자 정보가 'Speaker'로 시작하는지 확인
        if speaker_line.startswith('Speaker'):
            speaker_info = speaker_line.split()
            if len(speaker_info) >= 3:
                speaker = speaker_info[1]
                time = speaker_info[2]

                data.append([speaker, time, content_line])

        i += 2

    df = pd.DataFrame(data, columns=['Speaker', 'Time', 'Content'])
    return df

# 형태소 분석기를 사용한 전처리 함수
def tokenize_and_remove_stopwords(text):
    okt = Okt()

    tokens = okt.nouns(text)
    tokens = [token for token in tokens if token not in stopwords_list and len(token) > 1]
    return ' '.join(tokens)

file_path = 'transcriptions/project4/transcription_project4_09.txt'
df = load_and_preprocess(file_path)

# Content 열에 대해 전처리 수행
df['Processed_Content'] = df['Content'].apply(tokenize_and_remove_stopwords)

# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer(max_features=5)

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Processed_Content'])

# 키워드 확인
keywords = tfidf_vectorizer.get_feature_names_out()
print("키워드:", keywords)

# TF-IDF 행렬을 DataFrame으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=keywords)

# 원본 데이터와 결합
df = pd.concat([df, tfidf_df], axis=1)

# 각 참가자의 키워드 빈도 합계 계산
keyword_freq_by_speaker = df.groupby('Speaker')[keywords].sum()

# 중요 발언자 선정
important_speakers = keyword_freq_by_speaker.sum(axis=1).sort_values(ascending=False).index.tolist()

print("중요 발언자 순위:")
for speaker in important_speakers:
    print(f"{speaker}: {keyword_freq_by_speaker.loc[speaker].sum()}")

print("\n키워드 빈도 데이터프레임:")
print(keyword_freq_by_speaker)

키워드: ['그냥' '생각' '얘기' '이제' '지금']
중요 발언자 순위:
SPEAKER_03: 96.18115064516562
SPEAKER_00: 50.514220906525885
SPEAKER_01: 23.41154604699745
SPEAKER_04: 10.76984046696314
SPEAKER_02: 2.0

키워드 빈도 데이터프레임:
                   그냥         생각         얘기         이제         지금
Speaker                                                          
SPEAKER_00  10.000000  10.371457   6.540856  14.611388   8.990521
SPEAKER_01   9.000000   3.749183   2.000000   0.000000   8.662363
SPEAKER_02   1.000000   0.000000   0.000000   0.000000   1.000000
SPEAKER_03  16.480647   7.669751  18.315009  29.879340  23.836403
SPEAKER_04   3.552730   4.217110   1.000000   1.000000   1.000000


키워드 직접 입력 및 검색

In [4]:
import pandas as pd

# 데이터 로드 및 전처리
def load_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []

    i = 0
    while i < len(lines):
        speaker_line = lines[i].strip()
        content_line = lines[i+1].strip() if i+1 < len(lines) else ""

        # 발화자 정보가 'Speaker'로 시작하는지 확인
        if speaker_line.startswith('Speaker'):
            speaker_info = speaker_line.split()
            if len(speaker_info) >= 3:
                speaker = speaker_info[1]
                time = speaker_info[2]

                data.append([speaker, time, content_line])

        i += 2

    df = pd.DataFrame(data, columns=['Speaker', 'Time', 'Content'])
    return df

# 키워드 설정
keywords = ['협업', '외생변수', '녹음', 'HR', '부서를', '부서별', '시각화', '테스트', 'ab']

file_path = 'transcriptions/project4/transcription_project4_09.txt'
df = load_and_preprocess(file_path)

# 발언자별로 키워드 사용 횟수 계산
for keyword in keywords:
    df[keyword + '_Count'] = df['Content'].apply(lambda x: x.count(keyword))

# 각 발언자의 키워드 사용 횟수 합계 계산
keyword_counts = df.groupby('Speaker')[[keyword + '_Count' for keyword in keywords]].sum()

# 키워드별로 중요 발언자 선정
important_speakers = keyword_counts.sum(axis=1).sort_values(ascending=False).index.tolist()

print("중요 발언자 순위:")
for speaker in important_speakers:
    total_count = keyword_counts.loc[speaker].sum()
    print(f"{speaker}: {total_count} 회")

print("\n각 키워드 사용 횟수 데이터프레임:")
print(keyword_counts)

중요 발언자 순위:
SPEAKER_03: 10 회
SPEAKER_00: 9 회
SPEAKER_04: 7 회
SPEAKER_01: 1 회
SPEAKER_02: 1 회

각 키워드 사용 횟수 데이터프레임:
            협업_Count  외생변수_Count  녹음_Count  HR_Count  부서를_Count  부서별_Count  \
Speaker                                                                      
SPEAKER_00         1           1         1         1          0          4   
SPEAKER_01         0           1         0         0          0          0   
SPEAKER_02         0           0         1         0          0          0   
SPEAKER_03         2           1         3         0          0          1   
SPEAKER_04         0           0         2         1          0          1   

            시각화_Count  테스트_Count  ab_Count  
Speaker                                     
SPEAKER_00          1          0         0  
SPEAKER_01          0          0         0  
SPEAKER_02          0          0         0  
SPEAKER_03          0          1         2  
SPEAKER_04          0          2         1  


findall

In [6]:
import pandas as pd
import re

# 데이터 로드 및 전처리
def load_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []

    i = 0
    while i < len(lines):
        speaker_line = lines[i].strip()
        content_line = lines[i+1].strip() if i+1 < len(lines) else ""

        # 발화자 정보가 'Speaker'로 시작하는지 확인
        if speaker_line.startswith('Speaker'):
            speaker_info = speaker_line.split()
            if len(speaker_info) >= 3:
                speaker = speaker_info[1]
                time = speaker_info[2]

                data.append([speaker, time, content_line])

        i += 2

    df = pd.DataFrame(data, columns=['Speaker', 'Time', 'Content'])
    return df

# 키워드 설정
keywords = ['협업', '외생변수', 'HR', '부서별', '시각화', '테스트']

file_path = 'transcriptions/project4/transcription_project4_09.txt'
df = load_and_preprocess(file_path)

# 발언자별로 키워드 사용 횟수 계산 (정규표현식 사용)
for keyword in keywords:
    df[keyword + '_Count'] = df['Content'].apply(lambda x: len(re.findall(r'\b{}\b'.format(re.escape(keyword)), x, re.IGNORECASE)))

# 각 발언자의 키워드 사용 횟수 합계 계산
keyword_counts = df.groupby('Speaker')[[keyword + '_Count' for keyword in keywords]].sum()

# 키워드별로 중요 발언자 선정
important_speakers = keyword_counts.sum(axis=1).sort_values(ascending=False).index.tolist()

print("중요 발언자 순위:")
for speaker in important_speakers:
    total_count = keyword_counts.loc[speaker].sum()
    print(f"{speaker}: {total_count} 회")

print("\n각 키워드 사용 횟수 데이터프레임:")
print(keyword_counts)

중요 발언자 순위:
SPEAKER_00: 4 회
SPEAKER_03: 2 회
SPEAKER_04: 2 회
SPEAKER_01: 1 회
SPEAKER_02: 0 회

각 키워드 사용 횟수 데이터프레임:
            협업_Count  외생변수_Count  HR_Count  부서별_Count  시각화_Count  테스트_Count
Speaker                                                                    
SPEAKER_00         1           0         1          2          0          0
SPEAKER_01         0           1         0          0          0          0
SPEAKER_02         0           0         0          0          0          0
SPEAKER_03         0           1         0          0          0          1
SPEAKER_04         0           0         1          1          0          0
